# 1 - Instalando dependências

In [ ]:
# Instalando para usar
!pip install mediapipe opencv-python

In [ ]:
# Puxando elas

import cv2
# Itens do mp normalmente são chamados de soluções - solutions
import mediapipe as mp
import numpy as np
# mp_drawing nos da as utilidades de desenho dos objetos
mp_drawing = mp.solutions.drawing_utils
# Aqui a gente defini o modelo que vamos usar do mediapipe, pois existem vários, e vamos usar o para pose
mp_pose = mp.solutions.pose

In [ ]:
# Pegando a câmera

# Pegando o vídeo de alguma câmera conectada, a primeira - 0
cap = cv2.VideoCapture(0)

while cap.isOpened():
  # Ret é só o retorno, nada útil por agora
  # Frame é o frame mesmo, o que nos mostra
  ret, frame = cap.read()
  # Aqui o primeiro é o nome da caixa que temos o frame(vídeo), e o segundo o que mostramos
  cv2.imshow("Treinador de Academia Personalizado", frame)

  if cv2.waitKey(10) & 0xFF == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()

# 2 - Fazendo Detecções

In [ ]:
# Pegando o vídeo de alguma câmera conectada, a primeira - 0
cap = cv2.VideoCapture(0)

# Criando uma nova instância do pose do mediapipe definindo que o mínimo de confidência de detecção como 50%
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        # Ret é só o retorno, nada útil por agora
        # Frame é o frame mesmo, o que nos mostra
        ret, frame = cap.read()

        # Mudando a cor da imagem, pois fica melhor para o mediapipe, mandando na ordem certa, e transformando em RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Fazendo detecções
        results = pose.process(image)
        
        image.flags.writeable = True
        # Recolorindo para mostrar a imagem to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Rederizando as detecções
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                  # Mudando a cor dos pontos, primeiro dos pontos
                                  mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2), 
                                  # Aqui das conexões
                                  mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                                  )

        # Aqui o primeiro é o nome da caixa que temos o frame(vídeo), e o segundo o que mostramos
        cv2.imshow("Treinador de Academia Personalizado", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 3 - Determinando as juntas

Aqui recomendo ver a imagem das juntas que o mediapipe pega

In [ ]:
# Pegando o vídeo de alguma câmera conectada, a primeira - 0
cap = cv2.VideoCapture(0)

# Criando uma nova instância do pose do mediapipe definindo que o mínimo de confidência de detecção como 50%
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        # Ret é só o retorno, nada útil por agora
        # Frame é o frame mesmo, o que nos mostra
        ret, frame = cap.read()

        # Mudando a cor da imagem, pois fica melhor para o mediapipe, mandando na ordem certa, e transformando em RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Fazendo detecções
        results = pose.process(image)
        
        image.flags.writeable = True
        # Recolorindo para mostrar a imagem to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extraindo os pontos das juntas
        # Nem sempre dá certo e elas são visíveis, então fazemos dentro de um try para não quebrar
        try: 
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except: 
            print("Erro ao pegar as landmarks")
            pass

        # Rederizando as detecções
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                  # Mudando a cor dos pontos, primeiro dos pontos
                                  mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2), 
                                  # Aqui das conexões
                                  mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                                  )

        # Aqui o primeiro é o nome da caixa que temos o frame(vídeo), e o segundo o que mostramos
        cv2.imshow("Treinador de Academia Personalizado", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
landmarks

In [ ]:
len(landmarks)

In [ ]:
for landmarkExist in mp_pose.PoseLandmark:
    print(landmarkExist, landmarkExist.name)

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

In [ ]:
mp_pose.PoseLandmark.LEFT_SHOULDER

# 4 - Calculando ângulos

In [ ]:
def calculate_angle(a, b, c):
    a = np.array(a) # Primeiro - Ombro
    b = np.array(b) # Do meio - Cotovelo
    c = np.array(c) # Último - Mão

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    # Definindo o ângulo como máximo de 180, porque o braço não vai além
    if angle > 180.0:
        angle = 360 - angle

    return angle

In [ ]:
if landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility < 0.9:
    print("Pouca visibilidade do ombro, impossível calcular" , landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility)
else:
    print("Legal", landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility)

In [ ]:
# Pegando os valores da posição do ombro
shoulder = [
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
            ]

# Pegando os valores da posição do cotovelo
elbow = [
            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
            ]

# Pegando os valores da posição da mão
wrist = [
            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,
            ]

In [ ]:
print(shoulder)
print(elbow)
print(wrist)

In [ ]:
# Calculando o ângulo do braço
calculate_angle(shoulder, elbow, wrist)

In [ ]:
hipTest = [
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
            ]

shoulderTest = [
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
            ]

wristTest = [
            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
            ]

In [ ]:
print(hipTest, shoulderTest, wristTest)

In [ ]:
# Calculando a distância do corpo entre outros itens, apenas mostrando que da pra fazer
calculate_angle(hipTest, shoulderTest, wristTest)

In [ ]:
# Pegando as coordenadas normalizada pelo tamanho da imagem, nesse caso, com uma webcam de 640x480
tuple(np.multiply(elbow, [640, 480]).astype(int))

In [ ]:
# Pegando o tamanho da webcam de forma dinâmica em pixels para passar pro np.multiply
largura = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
altura = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

print(f"Resolução atual: {int(largura)}x{int(altura)}")

In [ ]:
# Pegando as posições do ângulo no vídeo e colocando no frame

# Pegando o vídeo de alguma câmera conectada, a primeira - 0
cap = cv2.VideoCapture(0)

# Criando uma nova instância do pose do mediapipe definindo que o mínimo de confidência de detecção como 50%
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        largura = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        altura = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

        print(f"Resolução atual: {int(largura)}x{int(altura)}")

        # Ret é só o retorno, nada útil por agora
        # Frame é o frame mesmo, o que nos mostra
        ret, frame = cap.read()

        # Mudando a cor da imagem, pois fica melhor para o mediapipe, mandando na ordem certa, e transformando em RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Fazendo detecções
        results = pose.process(image)
        
        image.flags.writeable = True
        # Recolorindo para mostrar a imagem to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extraindo os pontos das juntas
        # Nem sempre dá certo e elas são visíveis, então fazemos dentro de um try para não quebrar
        try: 
            landmarks = results.pose_landmarks.landmark

            # Pegando as coordenadas
            # Poderiamos fazer como fizemos a cima com outros pontos para calcular
            shoulder = [ landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                        ]
            elbow = [ landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                        ]
            wrist = [ landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,
                        ]
            
            # Calculando o ângulo
            angle = calculate_angle(shoulder, elbow, wrist)

            # Visualizando o ângulo
            cv2.putText(image, 
                str(angle), 
                tuple(np.multiply(elbow, [int(largura), int(altura)]).astype(int)), 
                # A partir daqui, apenas mexemos no texto
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.5, 
                (255, 255, 255), 
                2, 
                cv2.LINE_AA,
                )
        except: 
            print("Erro ao pegar as landmarks")
            pass

        # Rederizando as detecções
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                  # Mudando a cor dos pontos, primeiro dos pontos
                                  mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2), 
                                  # Aqui das conexões
                                  mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                                  )

        # Aqui o primeiro é o nome da caixa que temos o frame(vídeo), e o segundo o que mostramos
        cv2.imshow("Treinador de Academia Personalizado", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 4 - Criando o contador de curl

In [ ]:
# Pegando as posições do ângulo no vídeo e colocando no frame

# Pegando o vídeo de alguma câmera conectada, a primeira - 0
cap = cv2.VideoCapture(0)

# Variáveis para o contador
counter = 0
stage = None

# Criando uma nova instância do pose do mediapipe definindo que o mínimo de confidência de detecção como 50%
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        largura = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        altura = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

        # Ret é só o retorno, nada útil por agora
        # Frame é o frame mesmo, o que nos mostra
        ret, frame = cap.read()

        # Mudando a cor da imagem, pois fica melhor para o mediapipe, mandando na ordem certa, e transformando em RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Fazendo detecções
        results = pose.process(image)
        
        image.flags.writeable = True
        # Recolorindo para mostrar a imagem to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extraindo os pontos das juntas
        # Nem sempre dá certo e elas são visíveis, então fazemos dentro de um try para não quebrar
        try: 
            landmarks = results.pose_landmarks.landmark

            # Pegando as coordenadas
            # Poderiamos fazer como fizemos a cima com outros pontos para calcular
            shoulder = [ landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                        ]
            elbow = [ landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                        ]
            wrist = [ landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,
                        ]
            
            # Calculando o ângulo
            angle = calculate_angle(shoulder, elbow, wrist)

            # Visualizando o ângulo
            cv2.putText(image, 
                str(angle), 
                tuple(np.multiply(elbow, [int(largura), int(altura)]).astype(int)), 
                # A partir daqui, apenas mexemos no texto
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.5, 
                (255, 255, 255), 
                2, 
                cv2.LINE_AA,
                )
            
            # Criando a lógica para o contador
            if angle > 160:
                stage = "down"
            if angle < 30 and stage == "down":
                stage = "up"
                counter += 1
                print(counter)
        except: 
            print("Erro ao pegar as landmarks")
            pass

        # Renderizando o contador
        # Criando uma caixa
        cv2.rectangle(image, 
                      (0, 0), 
                      (225, 73), 
                      (245, 117, 16), 
                      -1,
                      )

        # Colocando os dados
        cv2.putText(image, 
                    "Reps", 
                    (15, 12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.5, 
                    (0, 0, 0), 
                    1, 
                    cv2.LINE_AA,
                    )
        cv2.putText(image, 
                    str(counter), 
                    (10, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    2, 
                    (255, 255, 255), 
                    2, 
                    cv2.LINE_AA,
                    )
        
        # Colocando o estado
        cv2.putText(image, 
                    "Estado", 
                    (65, 12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.5, 
                    (0, 0, 0), 
                    1, 
                    cv2.LINE_AA,
                    )
        cv2.putText(image, 
                    stage, 
                    (60, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    2, 
                    (255, 255, 255), 
                    2, 
                    cv2.LINE_AA,
                    )

        # Rederizando as detecções
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                  # Mudando a cor dos pontos, primeiro dos pontos
                                  mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2), 
                                  # Aqui das conexões
                                  mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                                  )

        # Aqui o primeiro é o nome da caixa que temos o frame(vídeo), e o segundo o que mostramos
        cv2.imshow("Treinador de Academia Personalizado", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()